In [3]:
from datetime import datetime
from concurrent import futures

import pandas as pd
import numpy as np
from pandas import DataFrame

import time,datetime
import glob

import math
import random as ran
from scipy.stats import truncnorm
import cmath
import re
import matplotlib.pyplot as plt
import pickle


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


from informs import portfolio
import class_informs as inform
import generator
import discriminator




In [4]:
with open('all_data.txt', 'rb') as f:
    risk_mat = pickle.load(f) 
    NEW_stocks = pickle.load(f) 
    close = pickle.load(f) 
    dic_alpha = pickle.load(f) 
    dic_bench = pickle.load(f) 
    dic_sector = pickle.load(f) 
    dic_MCAP = pickle.load(f) 
    THIS = pickle.load(f) 

    

In [6]:
bw=np.zeros(len(dic_bench))
alpha=np.zeros(len(dic_bench))

for i in range(len(NEW_stocks)):
    bw[i]=dic_bench[NEW_stocks[i]]
    alpha[i]=dic_alpha[NEW_stocks[i]]

sector_set = set(list(THIS['Sector']))
sector_list = list(THIS['Sector'])

gbs=THIS[['Sector', 'index']].apply(tuple, axis=1)
group_by_sector = [[y[1] for y in gbs if y[0]==x] for x in sector_set]

mq_list=list(THIS['mcapq'])
mq_set=set(list(THIS['mcapq']))

gbm=THIS[['mcapq', 'index']].apply(tuple, axis=1)
group_by_mq = [[y[1] for y in gbm if y[0]==x] for x in mq_set]

nbasset=len(bw)
nbsector=len(sector_set)
nbmq=len(mq_set)
X_dim = nbasset
HL_dim = 50
Z_dim = 10

car=50

np.random.seed()
G = generator.Generator(Z_dim, HL_dim, X_dim)

sample_D = discriminator.Discriminator(in_dim=X_dim, train=False, risk_mat=risk_mat, bw=bw, alpha=alpha, w_11=0.1
                                       ,c_lower=car)
train_D = discriminator.Discriminator(in_dim=X_dim, train=True, risk_mat=risk_mat, bw=bw, alpha=alpha, w_11=0.1
                                      ,c_lower=car)
G_solver = optim.Adam(G.parameters(), lr=2e-3)

mb_size = 128

best_obj = 100000.0
best_sols = []

losses = []

start_time = time.time()

feasible_threshold = 0.95

adaptive_inf_scale = float(nbasset)
rotation=1800

for it in range(rotation):
    if it % 900 == 0:
        sample_noise = generator.sample_Z(128, Z_dim)
        samples = G(sample_noise)

        sample_D.Zstar = best_obj
        sample_D.inf_scale = adaptive_inf_scale

        sample_D_values = sample_D(samples).data.numpy()
        sample_obj_values = sample_D.objs.data.numpy()

        fea_sample_obj_values = sample_obj_values[sample_D_values > feasible_threshold]
        sols = samples.data.numpy()[sample_D_values > feasible_threshold]

        if len(fea_sample_obj_values > 0):
            best_idx = np.argmin(fea_sample_obj_values)
            if np.min(fea_sample_obj_values) < best_obj:
                best_obj = np.minimum(fea_sample_obj_values[best_idx], best_obj)
                best_sols.append(sols[best_idx])

    z = Variable(torch.randn(mb_size, Z_dim))

    train_D.Zstar = best_obj
    train_D.inf_scale = adaptive_inf_scale

    G_loss = -torch.mean(torch.log(train_D(G(z))))

    G.zero_grad()
    G_loss.backward()
    G_solver.step()


best,bestsol=GAN_reulst(best_sols, sols,fea_sample_obj_values)
sol={}
for i in range(len(risk_sedol)):
    if i in best: sol[risk_sedol[i]]=1.
    else: sol[risk_sedol[i]]=0.


NameError: name 'GAN_reulst' is not defined

In [7]:
best_sols

[array([  5.76510502e-05,   3.67980610e-05,   6.35294637e-05,
          6.77745847e-05,   1.48306237e-04,   7.58944807e-05,
          1.66708400e-04,   3.26259178e-05,   3.67953908e-05,
          8.63753448e-05,   4.20501783e-05,   6.59131474e-05,
          1.53612345e-04,   5.05580501e-05,   9.01419771e-05,
          1.19333941e-04,   1.26235653e-04,   6.62125822e-05,
          1.74146364e-04,   3.44553082e-05,   9.75865260e-05,
          2.74435970e-05,   8.90221563e-05,   1.29283464e-04,
          7.87158060e-05,   8.67595372e-05,   3.63183462e-05,
          8.60089349e-05,   3.40351362e-05,   7.32220869e-05,
          3.39478102e-05,   5.33123239e-05,   1.04514919e-01,
          9.58850142e-05,   8.13936422e-05,   9.25858542e-02,
          5.36515872e-05,   6.68171851e-05,   1.20870553e-01,
          7.15793067e-05,   5.19091300e-05,   8.49745629e-05,
          9.49093550e-02,   9.01580497e-05,   6.58704230e-05,
          7.08949665e-05,   4.94539963e-05,   1.35085356e-04,
        